In [1]:
# import cupy as np
# when gpu, in colab

In [2]:
import numpy as np
# when cpu

In [3]:
from abc import ABC, abstractmethod

In [4]:
# pip install zope
# import zope.interface

---

### Activation Functions

In [5]:
# abstract class Function for creating activation function objects, to be used in different Layers
class Functions(ABC):
    # abstract method 'apply', used in forward passes
    @abstractmethod
    def apply(self, x):
        pass

    # abstract method 'gradient', used in backpropogation
    @abstractmethod
    def gradient(self, x):
        pass

In [6]:
# the default function for any layer, f(x) = x
class Basic(Functions):
    def apply(self, x):
        return x
    
    def gradient(self, x):
        if isinstance(x, np.darray):
            return np.ones(x.shape)
        return 1

In [7]:
# Sigmoid function
class Sigmoid(Functions):
    def apply(self, x):
        return 1 / (1 + np.exp(-x))
    
    def gradient(self, x):
        s = self.apply(x)
        return s*(1-s)

In [8]:
# Hyperbolic tangent function
class Tanh(Functions):
    def __init__(self, A = 1, S = 1):
        self.A = A
        self.S = S
        
    def apply(self, x):
        return self.A * np.tanh(self.S * x)
    
    def gradient(self, x):
        return (1 - (self.apply(x))**2) * self.S * self.A

---

### Models

In [9]:
# abstract class Layer, whih provides the basic structure to be followed all the different types of layers
class Layer(ABC):
    # ever child class must have an activation function, default is Basic
    def __init__(self, function = Basic()):
        self.function = function
        
    # abstract method, useful for initialising weights and biases according to the shapes
    # returns output shape of the layer
    @abstractmethod
    def initialise_weights(self, input_shape):
        pass

    # abstract method, useful for defining the forward calculations of the layer
    @abstractmethod
    def forward(self, x):
        pass

    # abstract method, useful for defining the backpropogation calculations and updating weights
    @abstractmethod
    def backward(self):
        pass

In [10]:
# Convolutional Layer
class Conv(Layer):
#    kernel shape - k - tuple of size 2 or 3, egs-> (3,3) or (3,3,2), 
#    output channels - c_out - int
#    and the activation function - function - Function
#    can be modified to add strides - s, if needed
    def __init__(self, k, c_out, function = Basic()):
        self.k = k
        self.c_out = c_out
        self.kernels = []
        super().__init__(function)
        # self.s = s

#    initialises c_out different kernels of shape k with uniformly distributed random weights
    def initialise_weights(self, input_shape):
        if len(self.k) == 2:
            h,w = self.k
            c = 0
        else:
            h,w,c = self.k
    
        for i in range(self.c_out):
            if (c == 0):
                self.kernels.append(np.random.rand(h,w))
            else:
                self.kernels.append(np.random.rand(h,w,c))

        return (input_shape[0] - self.k[0] + 1, 
                input_shape[1] - self.k[1] + 1, 
                self.c_out)

#    scratch implementation of the convolutional forward pass, with an activation function applied at the end
    def forward(self, x):
        new_h = x.shape[0] - self.k[0] + 1
        new_w = x.shape[1] - self.k[1] + 1

        # if self.c_out == 1:
        #   return convolve(x, self.kernels[0], new_h, new_w)

        out = np.zeros((new_h, new_w, self.c_out))
        for i in range(self.c_out):
            out[:,:,i] = self.convolve(x, i, new_h, new_w)

        return self.function.apply(out)

#    basic implementaion of a convolutional operation, used in forward pass
    def convolve(self, inp, kernel_no, new_h, new_w):
        kernel = self.kernels[kernel_no]
        out_c = np.zeros((new_h, new_w))

        for i in range(new_h):
            for j in range(new_w):
                out_c[i,j] = np.sum(inp[i : i + kernel.shape[0], j : j + kernel.shape[1]] * kernel)

        return out_c

#    to be implemented
    def backward(self):
        pass

In [11]:
# abstract parent class to implement Pooling
class Pool(Layer):
#    pool kernel shape - k - tuple of size 2
#    activation function - function
    def __init__(self, k, function = Basic()):
        self.k = k
        super().__init__(function)
        # self.s = s

#    normally pooling layers don't have trainable parameters, so doesn't initialise anything
#    returns output shape
    def initialise_weights(self, input_shape):
        if len(input_shape) == 2:
            return (input_shape[0] // self.k, input_shape[1] // self.k)

        return (input_shape[0] // self.k, input_shape[1] // self.k, 
                input_shape[2])
    
#    does a basic forward pass
    def forward(self, x):
        if len(x.shape) == 2:
            return self.pool_2d(x)

        out = np.zeros((x.shape[0] // self.k, x.shape[1] // self.k, x.shape[2]))
        for i in range(x.shape[2]):
            out[:,:,i] = self.pool_2d(x[:,:,i])

        return self.function.apply(out)

#    applies the specified pooling function, to a single channel
    def pool_2d(self, x_2d):
        out = np.zeros((x_2d.shape[0] // self.k, x_2d.shape[1] // self.k))
        for i in range(out.shape[0]):
            for j in range(out.shape[1]):
                h = i * self.k
                w = j * self.k
                out[i,j] = self.pooling_function(x_2d[h:h+self.k, w:w+self.k])

        return out

#    abstract method, lets children classes specify different pooling functions
    @abstractmethod
    def pooling_function(self, matrix):
        pass

#    to be implemented
    def backward(self):
        pass

In [12]:
# A pooling layer, with trainable parameters
class Subsampling(Pool):
    
#    initialises n different weights and biases which are random and uniformly distributed, n = input channels
    def initialise_weights(self, input_shape):
        n = 1 if len(input_shape) == 2 else input_shape[2]
        self.w = np.random.rand(n)
        self.b = np.random.rand(n)
#         print(input_shape)
        return super().initialise_weights(input_shape)

#    returns sum of the matrix
    def pooling_function(self, matrix):
        return np.sum(matrix)

#    forward pass, w * sum() + b
    def forward(self, x):
        out = super().forward(x)

        if len(out.shape) == 2:
            return out * self.w[0] + self.b[0]

        for i in range(out.shape[2]):
            out[:,:,i] = out[:,:,i] * self.w[i] + self.b[i]

        return out

In [13]:
# class MaxPool(Pool):
#     def __init__(self, k):
#         super().__init__(k)

#     def pooling_function(self, matrix):
#         return np.max(matrix)

In [14]:
# Flattens the input
class Flatten(Layer):
#    no weights in the picture, returns output shape
    def initialise_weights(self, input_shape):
        out_shape = 1
        for i in input_shape:
            out_shape *= i
        return (out_shape,1)

#    reshapes the input x to a column vector
    def forward(self, x):
        return x.reshape((-1,1))

#    to be implemented
    def backward(self):
        pass

In [15]:
# Fully Connected Layer
class Fully_Connected(Layer):
#    no. of neural units - units - int
#    activation function - function - Function
    def __init__(self, units, function = Basic()):
        self.units = units
        self.weights = None
        super().__init__(function)

#    initialises a matrix of weights, including biases, of shape (units, input + 1)
    def initialise_weights(self, input_shape):
        self.weights = np.random.rand(self.units, input_shape[0] + 1)
        return (self.units,1)

#    basic forward pass, returns units number of outputs
    def forward(self, x):
        x = x.reshape((-1,1))
        out = np.dot(self.weights, np.vstack((x,1)))
        return self.function.apply(out)

#    to be implemented
    def backward(self):
        pass

In [16]:
# a special layer, uses the euclidean distance between the input vector and its parameter vector
class RBF(Layer):
#    no. of neural units - units - int
#    activation function - function - Function
    def __init__(self, units, function = Basic()):
        self.units = units
        self.weights = None
        super().__init__(function)

#    initialises a matrix of weights, of shape (units, input)
    def initialise_weights(self, input_shape):
        self.weights = np.random.rand(self.units, input_shape[0])
        return (self.units,1)

#    does a basic forward pass, calcualtes the eulidean distances, and applies the activation function
    def forward(self, x):
        x = x.reshape((-1,1))
        out = self.weights - x.T
        out = np.square(out)
        out = np.sum(out, axis = 1)
        return self.function.apply(out)

#    to be implemented
    def backward(self):
        pass

---

### Model

In [17]:
# Model class, to define a model by adding layers, train and test data
class Model:
    def __init__(self):
        self.layers = []
        self.compiled = False

#    adds a layer to the list
    def add(self, layer):
        self.layers.append(layer)
        self.compiled = False

#    removes the layer at index
    def remove(self, index):
        self.list.pop(index)
        self.compiled = False

#    inialises the weights of all the layers and checks whether the output shape matches with the expected
    def compile(self, x_shape, y_shape, print_shapes = False):
        input_shape = x_shape
        for layer in self.layers:
            if print_shapes:
                print(input_shape, layer)
            input_shape = layer.initialise_weights(input_shape)

            if print_shapes:
                print("Output", input_shape)

        if input_shape == y_shape:
            print("The layers fit correctly")
            self.compiled = True
        else:
            print("The layers don't fit correctly")
            self.compiled = False
            
#    INCOMPLETE, backpropogation part is yet to be added
#    used for training the model
    def fit(self, x_train, y_train, epochs, validation_data):
        if not self.compiled:
            print("Compilation needed")
            return
            
        for epoch in range(epochs):
            layer_in = x_train
            outputs = [layer]

            for layer in self.layers:
                layer_in = layer.forward(layer_in)

            print("Model Trained")

#    runs the current model on the input dataset
#    used for testing the model
    def test(self, x_test):
        if not self.compiled:
            print("Compilation needed")
            return
        
        outputs = []

        for i in range(x_test.shape[0]):
            layer_in = x_test[i]
#             print(layer_in.shape)
            for layer in self.layers:
                layer_in = layer.forward(layer_in)
            outputs.append(layer_in)

        return np.array(outputs)

---